In [215]:
import pandas as pd
import numpy as np
import time
import datetime
from sklearn.cluster import DBSCAN
from sklearn.base import clone
from sklearn.utils import check_random_state
from sklearn.metrics import adjusted_rand_score, silhouette_score, normalized_mutual_info_score, adjusted_mutual_info_score

In [216]:
data = pd.read_csv('/Users/haikundu/Desktop/shanshu/hackathon/tidydata.csv')
data = data.fillna(0)
X = pd.DataFrame()

In [381]:
yyy = pd.read_csv('/Users/haikundu/Desktop/shanshu/hackathon/data/unit prod.csv')

In [382]:
yyy.columns

Index(['ProductionMonthKey', 'contactid', 'ProductionAmount',
       'NewRecruitCount', 'OrderingConsultantCount', 'NBCCount', 'N1BCCount',
       'N2BCCount', 'N3BCCount', 'ABCCount', 'A1BCCount', 'A2BCCount',
       'A3BCCount', 'PBCCount', 'P1BCCount', 'P2BCCount', 'P3BCCount',
       'TBCCount', 'T1BCCount', 'T2BCCount', 'T3BCCount', 'T4BCCount',
       'T5BCCount', 'T6BCCount', 'DIQCount', 'FDirCount', 'RJCount',
       'StarCount', 'SBCCount', 'BC15Count', 'BC10Count'],
      dtype='object')

## 标签设计

In [403]:
# label  
#choice 1 每个unit的总绩效
yyy = yyy[yyy['ProductionMonthKey']==201804] 
y1 = yyy[['ProductionAmount','contactid']]
#choice 2 每个unit的人均绩效
y2 = y1.copy()
y2['ProductionAmount'] = y2['ProductionAmount']/ (yyy['NBCCount'] + yyy['ABCCount'] + yyy['PBCCount'] + yyy['TBCCount'])

In [404]:
y2.to_csv('mean_y.csv')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, max_depth = 8)
r2 = cross_val_score(rf,x_train_scaled,y_train,cv=10,scoring = 'r2')
print(np.mean(r2))

## 特征设计

## Part 1

In [217]:
# feature 1 unit本身activity状态
X['activity_status'] = data.loc[:,'ActivityStatusCode_pprod1'].apply(lambda x: 4 if x == 'A1' else (3 if x == 'A2' else (2 if x == 'A3' else 1)))

In [218]:
# feature 2 职级
X['career_level'] = data.loc[:,'CareerLevelCode_pprod1']

In [219]:
# feature 3 司龄数字
X['age_MLK'] = data.loc[:,'monthssincebecomingbc_pprod1']

In [220]:
# feature 4-7 司龄one hot
X['age_MLK0'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x <= 12 else 0)
X['age_MLK1'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 12 and x <= 36 else 0)
X['age_MLK2'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 36 and x <= 60 else 0)
X['age_MLK3'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 60 else 0)

In [221]:
# feature 8 and 9 每个unit所在的地理位置（经纬度解析） 
X[['longitude','latitude']] = data.loc[:,['longitude','latitude']]

In [222]:
# feature 10 每个unit 的年龄
X['age'] = data['Birthday-year'].apply(lambda x: 2018-x)

In [223]:
# feature 11-14 年龄one hot
X['age0'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x <= 30 else 0)
X['age1'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 30 and x <= 40 else 0)
X['age2'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 40 and x <= 50 else 0)
X['age3'] = data['monthssincebecomingbc_pprod1'].apply(lambda x: 1 if x > 50 else 0)

In [224]:
# 地理位置 one hot 
onehot1 = pd.get_dummies(data['PrimaryCity'])

In [225]:
X = pd.concat([X,onehot1], axis = 1)

In [226]:
X.shape

(26403, 360)

## Part 2

In [41]:
# feature 1


(2321,)

## Part 3

In [227]:
# feature 1-5
X['personal_performance1'] = data.loc[:,'ProductionAmount_pprod1']
X['personal_performance2'] = data.loc[:,'ProductionAmount_pprod2']
X['personal_performance3'] = data.loc[:,'ProductionAmount_pprod3']
X['personal_performance6'] = data.loc[:,'ProductionAmount_pprod6']
X['personal_performance12'] = data.loc[:,'ProductionAmount_pprod12']

In [228]:
# feature 6-10
X['personal_order_rate1'] = data.loc[:,'OrderingConsultantIndicator_pprod1']
X['personal_order_rate2'] = data.loc[:,'OrderingConsultantIndicator_pprod2']
X['personal_order_rate3'] = (data['OrderingConsultantIndicator_pprod1'] + data['OrderingConsultantIndicator_pprod2'] + data['OrderingConsultantIndicator_pprod3'])/3
X['personal_order_rate6'] = (data['OrderingConsultantIndicator_pprod1'] + data['OrderingConsultantIndicator_pprod2'] + data['OrderingConsultantIndicator_pprod3'] + data['OrderingConsultantIndicator_pprod4'] + data['OrderingConsultantIndicator_pprod5'] + data['OrderingConsultantIndicator_pprod6'] )/6
X['personal_order_rate12'] = (data['OrderingConsultantIndicator_pprod1'] + data['OrderingConsultantIndicator_pprod2'] + data['OrderingConsultantIndicator_pprod3'] + data['OrderingConsultantIndicator_pprod4'] + data['OrderingConsultantIndicator_pprod5'] + data['OrderingConsultantIndicator_pprod6'] + data['OrderingConsultantIndicator_pprod7'] + data['OrderingConsultantIndicator_pprod8'] + data['OrderingConsultantIndicator_pprod9'] + data['OrderingConsultantIndicator_pprod10'] + data['OrderingConsultantIndicator_pprod11'] + data['OrderingConsultantIndicator_pprod12'] )/12

In [229]:
from sklearn.linear_model import LinearRegression
def get_scope(Y):
    X = np.array([i+1 for i in range(len(Y))]).reshape(-1,1)
    ols = LinearRegression().fit(X,Y).coef_
    return ols

In [230]:
# feature 11-20
X['personal_order_num'] = data.loc[:,'ProductionOrderCount_pprod1']
X['personal_order_sum3'] = data['ProductionOrderCount_pprod1'] + data['ProductionOrderCount_pprod2'] + data['ProductionOrderCount_pprod3']
X['personal_order_var3'] = data.apply(lambda x: np.var([x['ProductionOrderCount_pprod1'],x['ProductionOrderCount_pprod2'],x['ProductionOrderCount_pprod3']]),axis =1 )
X['personal_order_trend3'] = data.apply(lambda x: (x['ProductionOrderCount_pprod1']-x['ProductionOrderCount_pprod3'])/2 ,axis =1)

In [231]:
# feature 11-20
X['personal_order_sum6'] = data['ProductionOrderCount_pprod1'] + data['ProductionOrderCount_pprod2'] + data['ProductionOrderCount_pprod3'] + data['ProductionOrderCount_pprod4'] + data['ProductionOrderCount_pprod5'] + data['ProductionOrderCount_pprod6']
X['personal_order_var6'] = data.apply(lambda x: np.var([x['ProductionOrderCount_pprod1'],x['ProductionOrderCount_pprod2'],x['ProductionOrderCount_pprod3'],x['ProductionOrderCount_pprod4'],x['ProductionOrderCount_pprod5'],x['ProductionOrderCount_pprod6']]),axis =1 )
X['personal_order_trend6'] = data.apply(lambda x: (x['ProductionOrderCount_pprod1']-x['ProductionOrderCount_pprod6'])/5 ,axis =1)

In [232]:
# feature 11-20
trend12 = data[['ProductionOrderCount_pprod12','ProductionOrderCount_pprod11','ProductionOrderCount_pprod10','ProductionOrderCount_pprod9','ProductionOrderCount_pprod8','ProductionOrderCount_pprod7','ProductionOrderCount_pprod6','ProductionOrderCount_pprod5','ProductionOrderCount_pprod4','ProductionOrderCount_pprod3','ProductionOrderCount_pprod2','ProductionOrderCount_pprod1']]
trend12_y = []
for i in range(trend12.shape[0]):
    trend12_y.append(get_scope(trend12.loc[i,:]))
X['personal_order_sum12'] = trend12.sum(axis =1)
X['personal_order_var12'] = trend12.var(axis =1)
X['personal_order_trend12'] = trend12_y

In [233]:
# feature 21-24
act_status = pd.DataFrame()
act_status['A1'] = data['ActivityStatusCode_pprod1'].apply(lambda x: 3 if x == 'A1' else (2 if x == 'A2' else (1 if x == 'A3' else 0)))
act_status['A3'] = data['ActivityStatusCode_pprod3'].apply(lambda x: 3 if x == 'A1' else (2 if x == 'A2' else (1 if x == 'A3' else 0)))
act_status['A6'] = data['ActivityStatusCode_pprod6'].apply(lambda x: 3 if x == 'A1' else (2 if x == 'A2' else (1 if x == 'A3' else 0)))
act_status['A12'] = data['ActivityStatusCode_pprod12'].apply(lambda x: 3 if x == 'A1' else (2 if x == 'A2' else (1 if x == 'A3' else 0)))
X['activity_status1'] = act_status['A1']
X['activity_status3'] = act_status.apply(lambda x: 1 if x['A1'] > x['A3'] else (0 if x['A1'] == x['A3'] else -1),axis =1 )
X['activity_status6'] = act_status.apply(lambda x: 1 if x['A1'] > x['A6'] else (0 if x['A1'] == x['A6'] else -1),axis =1 )
X['activity_status12'] = act_status.apply(lambda x: 1 if x['A1'] > x['A12'] else (0 if x['A1'] == x['A12'] else -1),axis =1)

In [234]:
# feature 25-28
X['career_level1'] = data['CareerLevelCode_pprod1']
X['career_level3'] = data.apply(lambda x: 1 if x['CareerLevelCode_pprod1'] > x['CareerLevelCode_pprod3'] else (0 if x['CareerLevelCode_pprod1'] == x['CareerLevelCode_pprod3'] else -1),axis =1 )
X['career_level6'] = data.apply(lambda x: 1 if x['CareerLevelCode_pprod1'] > x['CareerLevelCode_pprod6'] else (0 if x['CareerLevelCode_pprod1'] == x['CareerLevelCode_pprod6'] else -1),axis =1 )
X['career_level12'] = data.apply(lambda x: 1 if x['CareerLevelCode_pprod1'] > x['CareerLevelCode_pprod12'] else (0 if x['CareerLevelCode_pprod1'] == x['CareerLevelCode_pprod12'] else -1),axis =1)

## Part 4

In [235]:
# feature 1 unit当前规模
X['scale'] = data.loc[:,'NBCCount1'] + data.loc[:,'ABCCount1'] + data.loc[:,'PBCCount1'] + data.loc[:,'TBCCount1']

In [236]:
# feature 2-3
X['unit_performance_sum1'] = data['ProductionAmount1']
X['unit_performance_avg1'] = data['ProductionAmount1'] / data['ABCCount1']

In [237]:
# feature 4-7
X['unit_performance_sum3'] = data['ProductionAmount1'] + data['ProductionAmount2'] + data['ProductionAmount3']
X['unit_performance_avg3'] = (data['ProductionAmount1'] / data['ABCCount1'] + data['ProductionAmount2'] / data['ABCCount2'] + data['ProductionAmount3'] / data['ABCCount3']) /3
X['unit_performance_var3'] = np.var([data['ProductionAmount1'] / data['ABCCount1'] , data['ProductionAmount2'] / data['ABCCount2'] , data['ProductionAmount3'] / data['ABCCount3']])
X['unit_performance_trend3'] = (data['ProductionAmount1']/data['ABCCount1'] - data['ProductionAmount3']/data['ABCCount3'])/2

In [238]:
# feature 8-11
X['unit_performance_sum6'] = data['ProductionAmount1'] + data['ProductionAmount2'] + data['ProductionAmount3'] + data['ProductionAmount4'] + data['ProductionAmount5'] + data['ProductionAmount6']
X['unit_performance_avg6'] = (data['ProductionAmount1'] / data['ABCCount1'] + data['ProductionAmount2'] / data['ABCCount2'] + data['ProductionAmount3'] / data['ABCCount3'] + data['ProductionAmount4'] / data['ABCCount4'] + data['ProductionAmount5'] / data['ABCCount5'] + data['ProductionAmount6'] / data['ABCCount6']) /6
X['unit_performance_var6'] = np.var([data['ProductionAmount1'] / data['ABCCount1'] , data['ProductionAmount2'] / data['ABCCount2'] , data['ProductionAmount3'] / data['ABCCount3'],data['ProductionAmount4'] / data['ABCCount4'] , data['ProductionAmount5'] / data['ABCCount5'] , data['ProductionAmount6'] / data['ABCCount6']])
X['unit_performance_trend6'] = (data['ProductionAmount1']/data['ABCCount1'] - data['ProductionAmount6']/data['ABCCount6'])/5

In [240]:
# feature 12-15
unit_perf12 = data[['ProductionAmount1','ProductionAmount2','ProductionAmount3','ProductionAmount4',
                    'ProductionAmount5','ProductionAmount6','ProductionAmount7','ProductionAmount8',
                    'ProductionAmount9','ProductionAmount10','ProductionAmount11','ProductionAmount12']]
unit_perf12_y = []
for i in range(unit_perf12.shape[0]):
    unit_perf12_y.append(get_scope(unit_perf12.loc[i,:]))
X['unit_performance_sum12'] = unit_perf12.sum(axis =1)
X['unit_performance_avg12'] = (data['ProductionAmount1'] / data['ABCCount1'] + data['ProductionAmount2'] / data['ABCCount2'] + data['ProductionAmount3'] / data['ABCCount3'] + data['ProductionAmount4'] / data['ABCCount4'] + data['ProductionAmount5'] / data['ABCCount5'] + data['ProductionAmount6'] / data['ABCCount6'] + data['ProductionAmount7'] / data['ABCCount7'] + data['ProductionAmount8'] / data['ABCCount8'] + data['ProductionAmount9'] / data['ABCCount9'] + data['ProductionAmount10'] / data['ABCCount10'] + data['ProductionAmount11'] / data['ABCCount11'] + data['ProductionAmount12'] / data['ABCCount12']) /12
X['unit_performance_var12'] = unit_perf12.var(axis =1)
X['unit_performance_trend12'] = unit_perf12_y

In [241]:
# feature 16-19
X['new_member1'] = data['NewRecruitCount1']
X['new_member3'] = data['NewRecruitCount3']
X['new_member6'] = data['NewRecruitCount6']
X['new_member12'] = data['NewRecruitCount12']

In [242]:
# feature 24-26
X['scale3'] = (data['NewRecruitCount1'] + data['NewRecruitCount2'] + data['NewRecruitCount3'])/3
X['scale6'] = (data['NewRecruitCount4'] + data['NewRecruitCount5'] + data['NewRecruitCount6'])/3
X['scale12'] = (data['NewRecruitCount7'] + data['NewRecruitCount8'] + data['NewRecruitCount9'] + data['NewRecruitCount10'] + data['NewRecruitCount11'] + data['NewRecruitCount12'])/6

In [243]:
# feature 27-34
act_num12 = data[['ABCCount1','ABCCount2','ABCCount3','ABCCount4',
                    'ABCCount5','ABCCount6','ABCCount7','ABCCount8',
                    'ABCCount9','ABCCount10','ABCCount11','ABCCount12']]
act_num12_y = []
for i in range(act_num12.shape[0]):
    act_num12_y.append(get_scope(act_num12.loc[i,:]))
    
X['active_number_num'] = data['ABCCount1']
X['active_number_rate1'] = data['ABCCount1'] / X['scale']
X['active_number_rate3'] = data['ABCCount3'] / (data['NBCCount3'] + data['ABCCount3'] + data['PBCCount3'] + data['TBCCount3'])
X['active_number_scale3'] = (data['ABCCount1'] - data['ABCCount3']) /2
X['active_number_rate6'] = data['ABCCount6'] / (data['NBCCount6'] + data['ABCCount6'] + data['PBCCount6'] + data['TBCCount6'])
X['active_number_scale6'] = (data['ABCCount1'] - data['ABCCount6']) /5
X['active_number_rate12'] = data['ABCCount12'] / (data['NBCCount12'] + data['ABCCount12'] + data['PBCCount12'] + data['TBCCount12'])
X['active_number_scale12'] = act_num12_y

In [244]:
# feature 35-39
X['order_member_num'] = data['OrderingConsultantCount1']
X['order_member_rate1'] = data['OrderingConsultantCount1'] / X['scale']
X['order_member_rate3'] = data['OrderingConsultantCount3'] / (data['NBCCount3'] + data['ABCCount3'] + data['PBCCount3'] + data['TBCCount3'])
X['order_member_rate6'] = data['OrderingConsultantCount6'] / (data['NBCCount6'] + data['ABCCount6'] + data['PBCCount6'] + data['TBCCount6'])
X['order_member_rate12'] = data['OrderingConsultantCount12'] / (data['NBCCount12'] + data['ABCCount12'] + data['PBCCount12'] + data['TBCCount12'])


In [245]:
# feature 41-51
X[['NBCCount1','ABCCount1','PBCCount1','TBCCount1',
   'DIQCount1','FDirCount1','RJCount1','StarCount1',
   'SBCCount1','BC10Count1','BC15Count1']] = data[['NBCCount1','ABCCount1','PBCCount1','TBCCount1',
   'DIQCount1','FDirCount1','RJCount1','StarCount1',
   'SBCCount1','BC10Count1','BC15Count1']]

In [246]:
# feature 52-72
X['DIQCount3'] = data['DIQCount3'] 
X['FDirCount3'] = data['FDirCount3'] 
X['RJCount3'] = data['RJCount3'] 
X['StarCount3'] = data['StarCount3']
X['SBCCount3'] = data['SBCCount3'] 
X['BC10Count3'] = data['BC10Count3'] 
X['BC15Count3'] = data['BC15Count3']
X['DIQCount6'] = data['DIQCount6'] 
X['FDirCount6'] = data['FDirCount6'] 
X['RJCount6'] = data['RJCount6'] 
X['StarCount6'] = data['StarCount6']
X['SBCCount6'] = data['SBCCount6'] 
X['BC10Count6'] = data['BC10Count6'] 
X['BC15Count6'] = data['BC15Count6']
X['DIQCount12'] = data['DIQCount12'] 
X['FDirCount12'] = data['FDirCount12'] 
X['RJCount12'] = data['RJCount12'] 
X['StarCount12'] = data['StarCount12']
X['SBCCount12'] = data['SBCCount12'] 
X['BC10Count12'] = data['BC10Count12'] 
X['BC15Count12'] = data['BC15Count12'] 

In [249]:
X['contactid'] = data['BK_ContactID']

In [401]:
X.shape

(26403, 456)

In [251]:
X.to_csv('part4.csv')  

In [252]:
relationship = pd.read_csv('/Users/haikundu/Desktop/shanshu/hackathon/data/unit relationships.csv')
relationship_12 = relationship[relationship['ProductionMonthKEY'] == 201803]

/Users/haikundu/anaconda/envs/python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [350]:
unit_info = relationship_12.groupby('parentcontactid').count().reset_index()[['parentcontactid','Affiliated']]
unit_info.columns = ['contactid','offspring']

child = relationship_12.loc[:,'offspringcontactid'].tolist()
parent = relationship_12.loc[:,'parentcontactid'].tolist()
edge = list(zip(parent,child))

import networkx as nx
DG = nx.DiGraph()
DG.add_edges_from(edge)
chief = [20002864080,20002867158,20002868550,20002946649,20002988314,
         20003048231,20003054633,20003059551,20003072734,20003081934,
         20003136657,20003153773,20003163023,20003207138,20003227384,
         20003239353,20003273865,20003284443,20003297686,20003301604,
         20003347433,20003391365,20003427590,20003456591,20003475059,
         20003508126,20003559602,20003681546,20003745973,20003832897,
         20003851927,20003886109,20003903683,20003919755,20003955501]

In [281]:
distance_chief = relationship_12[relationship_12.parentcontactid.isin(chief)][['offspringcontactid','OffspringLineNumber']]
combined = pd.merge(unit_info,distance_chief, left_on = 'contactid', right_on = 'offspringcontactid', how = 'left')[['contactid','offspring','OffspringLineNumber']]
combined['in_degree'] = combined['contactid'].apply(lambda x: DG.in_degree(x))

In [285]:
combined.head()

,contactid,offspring,OffspringLineNumber,in_degree
0,20002864080,4167,0.0,1
1,20002867158,1559,0.0,1
2,20002868550,6825,0.0,1
3,20002875680,2,1.0,2
4,20002883117,1,1.0,2


In [288]:
final = pd.merge(X,combined, left_on = 'contactid', right_on = 'contactid', how = 'left')

In [290]:
final.head()

,activity_status,career_level,age_MLK,age_MLK0,age_MLK1,age_MLK2,age_MLK3,longitude,latitude,age,...,FDirCount12,RJCount12,StarCount12,SBCCount12,BC10Count12,BC15Count12,contactid,offspring,OffspringLineNumber,in_degree
0,4,95,276,0,0,0,1,121.436525,31.188523,53,...,1.0,1.0,1.0,4.0,3.0,50.0,20002864080,4167,0.0,1
1,4,92,276,0,0,0,1,121.424624,31.220367,56,...,1.0,1.0,2.0,5.0,0.0,61.0,20002867158,1559,0.0,1
2,4,97,276,0,0,0,1,121.424624,31.220367,50,...,1.0,1.0,4.0,10.0,7.0,137.0,20002868550,6825,0.0,1
3,4,70,184,0,0,0,1,113.038017,28.137710,55,...,0.0,1.0,1.0,9.0,0.0,75.0,20002875680,2,1.0,2
4,4,60,196,0,0,0,1,119.988420,36.776358,37,...,0.0,1.0,0.0,2.0,0.0,28.0,20002883117,1,1.0,2


In [291]:
list(final.columns)

['activity_status',
 'career_level',
 'age_MLK',
 'age_MLK0',
 'age_MLK1',
 'age_MLK2',
 'age_MLK3',
 'longitude',
 'latitude',
 'age',
 'age0',
 'age1',
 'age2',
 'age3',
 '七台河',
 '万宁市',
 '三亚',
 '三明',
 '三门峡',
 '上海',
 '上饶',
 '东方市',
 '东莞',
 '东营',
 '中卫',
 '中山',
 '临夏回族自治州',
 '临汾',
 '临沂',
 '临沧',
 '丹东',
 '丽水',
 '丽江',
 '乌兰察布',
 '乌海',
 '乌鲁木齐',
 '乐山',
 '九江',
 '云浮',
 '五家渠',
 '亳州',
 '仙桃市',
 '伊犁哈萨克自治州',
 '佛山',
 '佳木斯',
 '保亭黎族苗族自治县',
 '保定',
 '保山',
 '信阳',
 '儋州',
 '克孜勒苏柯尔克孜自治州',
 '克拉玛依',
 '六安',
 '六盘水',
 '兰州',
 '兴安盟',
 '内江',
 '凉山彝族自治州',
 '包头',
 '北京',
 '北海',
 '十堰',
 '南京',
 '南充',
 '南宁',
 '南平',
 '南昌',
 '南通',
 '南阳',
 '博尔塔拉蒙古自治州',
 '厦门',
 '双鸭山',
 '台州',
 '合肥',
 '吉安',
 '吉林',
 '吐鲁番地区',
 '吕梁',
 '吴忠',
 '周口',
 '呼伦贝尔',
 '呼和浩特',
 '和田地区',
 '咸宁',
 '咸阳',
 '哈密地区',
 '哈尔滨',
 '唐山',
 '商丘',
 '商洛',
 '喀什地区',
 '嘉兴',
 '嘉峪关',
 '四平',
 '固原',
 '塔城地区',
 '大兴安岭地区',
 '大同',
 '大庆',
 '大理白族自治州',
 '大连',
 '天水',
 '天津',
 '天门',
 '太原',
 '威海',
 '娄底',
 '孝感',
 '宁德',
 '宁波',
 '安庆',
 '安康',
 '安阳',
 '安顺',
 '定西',
 '宜宾',
 '宜昌',
 '宜春',
 '宝鸡',
 '宣城',
 '宿州'

In [351]:
bfs = bfs_edges(DG,contactid[5])
#unit_member = [x[1] for x in list(bfs)]
#temp_info_continuous = X[X.contactid.isin(unit_member)]
#temp_info_discrete = X[X.contactid.isin(unit_member)]

In [353]:
DG.out_degree(contactid[5])

1

In [318]:
from networkx.algorithms.traversal.breadth_first_search import bfs_edges

def find_unit_info(id):
    bfs = bfs_edges(DG,id)
    unit_member = [x[1] for x in list(bfs)]
    temp_info_continuous = X[X.contactid.isin(unit_member)][['latitude','longitude','age','age_MLK']]
    temp_info_discrete = X[X.contactid.isin(unit_member)][['age0','age1','age2','age3',
     'age_MLK0',
     'age_MLK1',
     'age_MLK2',
     'age_MLK3',
     '七台河',
     '万宁市',
     '三亚',
     '三明',
     '三门峡',
     '上海',
     '上饶',
     '东方市',
     '东莞',
     '东营',
     '中卫',
     '中山',
     '临夏回族自治州',
     '临汾',
     '临沂',
     '临沧',
     '丹东',
     '丽水',
     '丽江',
     '乌兰察布',
     '乌海',
     '乌鲁木齐',
     '乐山',
     '九江',
     '云浮',
     '五家渠',
     '亳州',
     '仙桃市',
     '伊犁哈萨克自治州',
     '佛山',
     '佳木斯',
     '保亭黎族苗族自治县',
     '保定',
     '保山',
     '信阳',
     '儋州',
     '克孜勒苏柯尔克孜自治州',
     '克拉玛依',
     '六安',
     '六盘水',
     '兰州',
     '兴安盟',
     '内江',
     '凉山彝族自治州',
     '包头',
     '北京',
     '北海',
     '十堰',
     '南京',
     '南充',
     '南宁',
     '南平',
     '南昌',
     '南通',
     '南阳',
     '博尔塔拉蒙古自治州',
     '厦门',
     '双鸭山',
     '台州',
     '合肥',
     '吉安',
     '吉林',
     '吐鲁番地区',
     '吕梁',
     '吴忠',
     '周口',
     '呼伦贝尔',
     '呼和浩特',
     '和田地区',
     '咸宁',
     '咸阳',
     '哈密地区',
     '哈尔滨',
     '唐山',
     '商丘',
     '商洛',
     '喀什地区',
     '嘉兴',
     '嘉峪关',
     '四平',
     '固原',
     '塔城地区',
     '大兴安岭地区',
     '大同',
     '大庆',
     '大理白族自治州',
     '大连',
     '天水',
     '天津',
     '天门',
     '太原',
     '威海',
     '娄底',
     '孝感',
     '宁德',
     '宁波',
     '安庆',
     '安康',
     '安阳',
     '安顺',
     '定西',
     '宜宾',
     '宜昌',
     '宜春',
     '宝鸡',
     '宣城',
     '宿州',
     '宿迁',
     '屯昌县',
     '岳阳',
     '崇左',
     '巢湖',
     '巴中',
     '巴彦淖尔',
     '巴音郭楞蒙古自治州',
     '常州',
     '常德',
     '平凉',
     '平顶山',
     '广元',
     '广安',
     '广州',
     '庆阳',
     '廊坊',
     '延安',
     '延边朝鲜族自治州',
     '开封',
     '张家口',
     '张家界',
     '张掖',
     '徐州',
     '德宏傣族景颇族自治州',
     '德州',
     '德阳',
     '忻州',
     '怀化',
     '怒江傈僳族自治州',
     '恩施',
     '惠州',
     '成都',
     '扬州',
     '承德',
     '抚州',
     '抚顺',
     '揭阳',
     '攀枝花',
     '文山壮族苗族自治州',
     '文昌市',
     '新乡',
     '新余',
     '无锡',
     '日照',
     '昆明',
     '昌吉回族自治州',
     '昌江黎族自治县',
     '昭通',
     '晋中',
     '晋城',
     '普洱',
     '景德镇',
     '曲靖',
     '朔州',
     '朝阳',
     '本溪',
     '来宾',
     '杭州',
     '松原',
     '枣庄',
     '柳州',
     '株洲',
     '桂林',
     '梅州',
     '梧州',
     '楚雄彝族自治州',
     '榆林',
     '武威',
     '武汉',
     '毕节',
     '永州',
     '汉中',
     '汕头',
     '汕尾',
     '江门',
     '池州',
     '沈阳',
     '沧州',
     '河池',
     '河源',
     '泉州',
     '泰安',
     '泰州',
     '泸州',
     '洛阳',
     '济南',
     '济宁',
     '海东地区',
     '海北藏族自治州',
     '海南藏族自治州',
     '海口',
     '海西蒙古族藏族自治州',
     '淄博',
     '淮北',
     '淮南',
     '淮安',
     '深圳',
     '清远',
     '温州',
     '渭南',
     '湖州',
     '湘潭',
     '湘西土家族苗族自治州',
     '湛江',
     '滁州',
     '滨州',
     '漯河',
     '漳州',
     '潍坊',
     '潜江',
     '潮州',
     '澄迈县',
     '濮阳',
     '烟台',
     '焦作',
     '牡丹江',
     '玉林',
     '玉树藏族自治州',
     '玉溪',
     '珠海',
     '琼中黎族苗族自治县',
     '琼海市',
     '甘南藏族自治州',
     '甘孜藏族自治州',
     '白城',
     '白山',
     '白银',
     '百色',
     '益阳',
     '盐城',
     '盘锦',
     '眉山',
     '石嘴山',
     '石家庄',
     '石河子',
     '福州',
     '秦皇岛',
     '红河哈尼族彝族自治州',
     '绍兴',
     '绥化',
     '绵阳',
     '聊城',
     '肇庆',
     '自贡',
     '舟山',
     '芜湖',
     '苏州',
     '茂名',
     '荆州',
     '荆门',
     '莆田',
     '莱芜',
     '菏泽',
     '萍乡',
     '营口',
     '葫芦岛',
     '蚌埠',
     '衡水',
     '衡阳',
     '衢州',
     '襄阳',
     '西双版纳傣族自治州',
     '西宁',
     '西安',
     '许昌',
     '贵港',
     '贵阳',
     '贺州',
     '资阳',
     '赣州',
     '赤峰',
     '辽源',
     '辽阳',
     '达州',
     '运城',
     '连云港',
     '迪庆藏族自治州',
     '通化',
     '通辽',
     '遂宁',
     '遵义',
     '邢台',
     '邯郸',
     '邵阳',
     '郑州',
     '郴州',
     '鄂尔多斯',
     '鄂州',
     '酒泉',
     '重庆',
     '金华',
     '金昌',
     '钦州',
     '铁岭',
     '铜仁',
     '铜川',
     '铜陵',
     '银川',
     '锡林郭勒盟',
     '锦州',
     '镇江',
     '长春',
     '长沙',
     '长治',
     '阜新',
     '阜阳',
     '防城港',
     '阳江',
     '阳泉',
     '阿克苏地区',
     '阿勒泰地区',
     '阿坝藏族羌族自治州',
     '阿拉善盟',
     '阿拉尔市',
     '陇南',
     '陵水黎族自治县',
     '随州',
     '雅安',
     '青岛',
     '鞍山',
     '韶关',
     '马鞍山',
     '驻马店',
     '鸡西',
     '鹤壁',
     '鹤岗',
     '鹰潭',
     '黄冈',
     '黄南藏族自治州',
     '黄山',
     '黄石',
     '黑河',
     '黔东南苗族侗族自治州',
     '黔南州',
     '黔南布依族苗族自治州',
     '黔西南布依族苗族自治州',
     '齐齐哈尔',
     '龙岩',]]
    temp_mean = temp_info_continuous.mean()
    temp_var = temp_info_continuous.var()
    temp_sum = temp_info_discrete.sum()
    temp_all = temp_mean.append(temp_var)
    temp_all = temp_all.append(temp_sum)
    return temp_all


In [358]:
contactid = X['contactid'].values
start = find_unit_info(contactid[0])
for i in range(1,len(contactid)):
    new_info = find_unit_info(contactid[i])
    start = pd.concat([start,new_info],axis =1)
final_info = start.T

In [366]:
list(final_info.columns)

['latitude',
 'longitude',
 'age',
 'age_MLK',
 'latitude',
 'longitude',
 'age',
 'age_MLK',
 'age0',
 'age1',
 'age2',
 'age3',
 'age_MLK0',
 'age_MLK1',
 'age_MLK2',
 'age_MLK3',
 '七台河',
 '万宁市',
 '三亚',
 '三明',
 '三门峡',
 '上海',
 '上饶',
 '东方市',
 '东莞',
 '东营',
 '中卫',
 '中山',
 '临夏回族自治州',
 '临汾',
 '临沂',
 '临沧',
 '丹东',
 '丽水',
 '丽江',
 '乌兰察布',
 '乌海',
 '乌鲁木齐',
 '乐山',
 '九江',
 '云浮',
 '五家渠',
 '亳州',
 '仙桃市',
 '伊犁哈萨克自治州',
 '佛山',
 '佳木斯',
 '保亭黎族苗族自治县',
 '保定',
 '保山',
 '信阳',
 '儋州',
 '克孜勒苏柯尔克孜自治州',
 '克拉玛依',
 '六安',
 '六盘水',
 '兰州',
 '兴安盟',
 '内江',
 '凉山彝族自治州',
 '包头',
 '北京',
 '北海',
 '十堰',
 '南京',
 '南充',
 '南宁',
 '南平',
 '南昌',
 '南通',
 '南阳',
 '博尔塔拉蒙古自治州',
 '厦门',
 '双鸭山',
 '台州',
 '合肥',
 '吉安',
 '吉林',
 '吐鲁番地区',
 '吕梁',
 '吴忠',
 '周口',
 '呼伦贝尔',
 '呼和浩特',
 '和田地区',
 '咸宁',
 '咸阳',
 '哈密地区',
 '哈尔滨',
 '唐山',
 '商丘',
 '商洛',
 '喀什地区',
 '嘉兴',
 '嘉峪关',
 '四平',
 '固原',
 '塔城地区',
 '大兴安岭地区',
 '大同',
 '大庆',
 '大理白族自治州',
 '大连',
 '天水',
 '天津',
 '天门',
 '太原',
 '威海',
 '娄底',
 '孝感',
 '宁德',
 '宁波',
 '安庆',
 '安康',
 '安阳',
 '安顺',
 '定西',
 '宜宾',
 '宜昌',
 '宜春',
 '宝鸡',
 '

In [363]:
final_fullinfo = final_info.fillna(0)

In [377]:
final_fullinfo['contactid'] = contactid

In [373]:
X.shape

(26403, 456)

In [378]:
output = pd.merge(X,final_fullinfo,left_on = 'contactid' ,right_on = 'contactid' ,how = 'left' )

In [380]:
output.to_csv('final.csv')

,activity_status,career_level,age_MLK,age_MLK0,age_MLK1,age_MLK2,age_MLK3,longitude,latitude,age,...,FDirCount12,RJCount12,StarCount12,SBCCount12,BC10Count12,BC15Count12,contactid,offspring,OffspringLineNumber,in_degree
count,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,...,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,26403.000000,2.640300e+04,26403.000000,26403.000000,26403.000000
mean,3.685718,64.711321,92.403288,0.012915,0.133356,0.193425,0.660304,114.057728,31.742233,38.940537,...,0.202098,0.202856,0.439874,2.382722,0.743703,27.637806,2.000781e+10,5.889975,5.179904,5.923494
std,0.620402,6.709355,51.900123,0.112911,0.339966,0.394991,0.473615,6.766254,5.835228,7.320776,...,0.522489,0.510013,0.821721,2.867577,1.972509,22.777905,3.420881e+06,86.127368,1.898128,1.851193
min,1.000000,60.000000,7.000000,0.000000,0.000000,0.000000,0.000000,75.259228,18.252847,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000286e+10,1.000000,0.000000,1.000000
25%,4.000000,60.000000,49.000000,0.000000,0.000000,0.000000,0.000000,111.439091,28.116083,33.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,2.000494e+10,1.000000,4.000000,5.000000
50%,4.000000,60.000000,85.000000,0.000000,0.000000,0.000000,1.000000,114.385539,31.247105,38.000000,...,0.000000,0.000000,0.000000,2.000000,0.000000,25.000000,2.000712e+10,1.000000,5.000000,6.000000
75%,4.000000,70.000000,130.000000,0.000000,0.000000,0.000000,1.000000,118.796877,35.420364,44.000000,...,0.000000,0.000000,1.000000,4.000000,1.000000,37.000000,2.001014e+10,2.000000,6.000000,7.000000
max,4.000000,97.000000,276.000000,1.000000,1.000000,1.000000,1.000000,134.013872,52.334457,74.000000,...,9.000000,7.000000,13.000000,37.000000,60.000000,549.000000,2.001586e+10,8060.000000,13.000000,14.000000


In [402]:
from sklearn.preprocessing import scale, StandardScaler
scaler = StandardScaler()
clusters = scaler.fit_transform(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(min_samples = 10,eps = 0.001)
clusters = dbscan.fit_predict(X)

In [ ]:
set(clusters)

In [ ]:
silhouette = []
eps_range = np.logspace(-3, -0.8, 6)
for eps in eps_range:
    dbscan = DBSCAN(eps=eps)
    dbscan.fit(data)
    silhouette.append(silhouette_score(data, dbscan.labels_))
    
st, = plt.twinx().plot(range(6), silhouette, label="silhouette")
plt.yticks(())

plt.legend(st, "Silhouette")
plt.xlabel("index")
plt.title("Scanning eps with different scores")

In [ ]:
set(clusters)

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10, random_state=0)
km.fit(data)
kmeans_pred = km.predict(data)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib import colors
#cmap = colors.ListedColormap(['#0096B3','#b35200','#35deff','#ff9942'])
X_tsne = TSNE(perplexity = 10).fit_transform(X)
plt.scatter(X_tsne[:,0], X_tsne[:,1], c = kmeans_pred ,alpha = 0.5)

In [ ]:
kmeans_pred

In [ ]:
# feature 3 每个unit的司龄
def Caltime(date):
    date = time.strptime(date,"%Y-%m-%d")
    today = time.strptime('2018-06-04',"%Y-%m-%d")
    date = datetime.datetime(date[0],date[1],date[2])  
    today = datetime.datetime(today[0],today[1],today[2])
    return (today - date).days

X['age_MLK'] = data['StartDate'].apply(lambda x: Caltime(x))